# iLQR
Location for Linear-Quadratic Regulator (LQR) and iLQR for interactive trajectory planning.

References:
- [1] - [Dewolf - studywolf_control](https://github.com/studywolf/control/blob/master/studywolf_control/controllers/ilqr.py)
- [2] - [Fridovich-Keil - ilqgames](https://github.com/HJReachability/ilqgames/blob/master/python/dynamical_system.py)
- [3] - [Jackson, Howell - iLQR Tutorial](http://roboticexplorationlab.org/papers/iLQR_Tutorial.pdf)
- [4] - [Jackson - AL iLQR Tutorial](https://bjack205.github.io/papers/AL_iLQR_Tutorial.pdf)
- [5] - [Sears-Collins - Linear Quadratic Regulator Python Example](https://automaticaddison.com/linear-quadratic-regulator-lqr-with-python-code-example/)

In [1]:
import sys
sys.path.append('/mnt/c/Users/ZachJW/Documents/Grad/Research/icon/icon/')

import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import solve_discrete_are

from control import iLQR, LQR
from models import DoubleInt1dDynamics, DoubleInt2dDynamics
from models import CarDynamics, UnicycleDynamics, BicycleDynamics
from models import CarDynamicsDiff, UnicycleDynamicsDiff, BicycleDynamicsDiff
from cost import ObstacleCost, ReferenceCost
from util import Point

In [5]:
cost = ObstacleCost((0, 1), Point(10, 10), 10)
x = np.array([0, 0, 0, 0])
u = np.array([0, 0])
L_x = cost.quadraticize(x, None)
L_x

array([-20., -20.,   0.,   0.])

In [7]:
n_x = x.shape[0]
n_u = u.shape[0]

state_mask = np.zeros((n_x))
state_mask[[cost.x_ind, cost.y_ind]] = -1

x_o = np.zeros((n_x))
x_o[cost.x_ind] = cost.point.x
x_o[cost.y_ind] = cost.point.y

L_x = 2*(x*state_mask - x_o).T
L_u = np.zeros((n_u))
L_xx = 2*np.diag(state_mask)
L_xx

array([[-2.,  0.,  0.,  0.],
       [ 0., -2.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.]])

In [4]:
%matplotlib ipympl
np.set_printoptions(precision=3, suppress=True)

In [4]:
plt.figure()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### iLQR

dt = 0.01
N = 100
n_lqr_iter = 60
tol = 1e-6
print('finite difference')

# dynamics = CarDynamics(dt)
dynamics = CarDynamicsDiff(dt)
x0 = np.array([-1, 2, np.pi/4])
xf = np.array([0, 0, 0])
Q = np.diag([1, 1, 0])
R = np.eye(2) * 0.1

Qf = np.eye(Q.shape[0]) * 1e3
quad_cost = ReferenceCost(xf, Q, R, Qf)
ilqr = iLQR(dynamics, quad_cost, N=N)
X, U, cost = ilqr.run(x0, n_lqr_iter, tol)
dynamics.plot(X, quad_cost.xf, False)

In [5]:
dt = 0.1
N = 60
n_lqr_iter = 60
tol = 1e-6

# dynamics = DoubleInt1dDynamics(dt)
# x0 = np.array([2, 0])
# xf = np.array([0, 1])
# Q = np.diag([1, 1])
# R = np.eye(1) * 0.01

# dynamics = DoubleInt2dDynamics(dt)
# x0 = np.array([1, 1, 10, 0])
# xf = np.array([0, 0, 0, 0])
# Q = np.diag([1, 1, .1, .1])
# R = np.eye(2) * 0.01

# dynamics = CarDynamics(dt)
# dynamics = CarDynamicsDiff(dt)
# x0 = np.array([-1, -1, np.pi/2])
# xf = np.array([0, 0, 0])
# Q = np.diag([1, 1, 0])
# R = np.eye(2)

# dynamics = UnicycleDynamics(dt)
# dynamics = UnicycleDynamicsDiff(dt)
# x0 = np.array([1, -2, 0, 0])
# xf = np.array([0, 0, 0, 0])
# Q = np.diag([1, 1, 0, 0])
# R = np.eye(2)

# dynamics = BicycleDynamics(dt)
dynamics = BicycleDynamicsDiff(dt)
x0 = np.array([-1, 1, np.pi/2, 0, 0])
xf = np.array([0, 0, 0, 0, 0])
Q = np.diag([1, 1, 0, 0, 0])
R = np.eye(2)

Qf = np.eye(Q.shape[0]) * 1e3
cost = ReferenceCost(xf, Q, R, Qf)
ilqr = iLQR(dynamics, cost, N=N)
X, U, Jf = ilqr.run(x0, n_lqr_iter, tol)
dynamics.plot(X, cost.xf, Jf, False)

[run] 0/60	J: 4587.4
[run] 1/60	J: 2154.82	μ: 0.5	Δ: 0.5
[run] 2/60	J: 111.067	μ: 0.125	Δ: 0.25
[run] 3/60	J: 52.8729	μ: 0.015625	Δ: 0.125
[run] 4/60	J: 52.3484	μ: 0.000976562	Δ: 0.0625
[run] 5/60	J: 52.329	μ: 3.05176e-05	Δ: 0.03125
[run] 6/60	J: 52.3277	μ: 0	Δ: 0.015625
[run] 7/60	J: 52.3275	μ: 0	Δ: 0.0078125


In [18]:
plt.gca().axis([-2.1, 0.3, -1.0, 0.6])

(-2.1, 0.3, -1.0, 0.6)

### LQR

In [6]:
N = 60
dt = 0.1

# dynamics = DoubleInt1dDynamics(dt)
# x0 = np.array([10, 0])
# xf = np.array([0, 0])
# Q = np.diag([1, 1])
# R = np.eye(1)

dynamics = DoubleInt2dDynamics(dt)
x0 = np.array([1, 1, 0, 0])
xf = np.array([0, 0, 0, 0])
Q = np.eye(4)
R = np.eye(2)

# dynamics = CarDynamicsDiff(dt)
# x0 = np.array([5.0, 5.0, np.pi/2])
# xf = np.array([0, 0, 0])
# Q = np.diag([1, 1, 0])
# R = np.eye(2) * 0.01

# dynamics = UnicycleDynamicsDiff(dt)
# x0 = np.array([-1, -1, 0, np.pi/2])
# xf = np.array([0, 0, 0, 0])
# Q = np.diag([1, 1, 0, 0])
# R = np.eye(2)

Qf = np.eye(Q.shape[0]) * 1e3
quad_cost = ReferenceCost(xf, Q, R, Qf)
lqr = LQR(dynamics, quad_cost, N=N)
X, U, J = lqr.run(x0)
dynamics.plot(X, xf, J)

J0: 2.11e+03	Jf: 36.3
